# 먼저 라이브러리들을 사용해서 이진 분류 해보자

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
#데이터 불러오기
df = pd.read_csv('binary_dataset.csv')
#df

In [ ]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


In [ ]:
df.shape

(20, 9)

In [ ]:
features = df.drop(columns = 'y')
label = df.iloc[:,8]

In [ ]:
# 학습데이터는 0.8 테스트 데이터는 0.2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(16, 8)
(4, 8)


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(8,)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1)

1/1 [==============================] - 2s 2s/step - loss: 21.2685 - accuracy: 0.6250


In [ ]:
model.evaluate(X_test,  y_test, verbose=2)

1/1 - 0s - loss: 39.4972 - accuracy: 0.5000 - 142ms/epoch - 142ms/step


[39.497222900390625, 0.5]

# 라이브러리를 안쓰는 프로젝트

In [1]:
#얘네만 사용하기

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [2]:
#데이터 불러오기
def load_data():
  global df
  urls = 'binary_dataset.csv'
  df = pd.read_csv(urls)

In [3]:
#load_data()


In [4]:
#파라미터와 편향 생성하기
#가중치 초기화는 표준정규분포에서 뽑기
#원래 기울기 초기화의 경우 역전파에서 가중치 소실이나 폭발이 일어나서 xavier나 he를 쓰지만 여기서는 역전파가 없으니 그냥 초기화
#편향의 경우 0으로 초기화하거나 0.01로 초기화하는 경우가 많아서 0.01로 

def para():
  global weight
  global bias
  weight = np.random.randn(8)
  bias     = 0.01


In [5]:
#para()
#weight,bias

In [6]:
#데이터 셔플해서 인덱스만 다시 설정하는 메서드


def data_shu():
  global df_shu
  df_shu = df.sample(frac=1).reset_index(drop = True)
  


In [7]:
#df_shu = data_shu(df) #기존의 df랑 다른 것을 확인
#data_shu()

In [8]:
#학습데이터와 테스트 데이터를 0.8 0.2 로 나누기
#20개중에서 무작위로 4개를 뽑고 얘는 테스트 데이터로 삼자
#매개변수를 test_size = 0.2로 했을때 그것만 뽑으려면 (전체 데이터 개수 * test_size매개변수로 해서 그것만 뽑자)


def data_split():
  li = []
  test_size = 0.2
  start = 0
  end   = df_shu.shape[0] 
  test_ = df_shu.shape[0]*test_size

  for _ in range(int(test_)):
    a = np.random.randint(start,end, size=1)[0]
    while a in li:
      a = np.random.randint(start,end, size=1)[0]
    li.append(a)
  global X_train
  global X_test
  global y_train
  global y_test

  X_test,y_test   = df_shu.iloc[li,:8],df_shu.iloc[li,8]
  X_train,y_train = df_shu.drop(index = li).iloc[:,:8],df_shu.drop(index = li).iloc[:,8]






In [9]:
#data_split()


In [10]:
#미니 배치로 데이터 나누기

def mini_batch():
  #4개씩 x_train을 나누기
  
  global mini_1,mini_2,mini_3,mini_4
  
  mini_1 = X_train.iloc[0:4,:]
  mini_2 = X_train.iloc[4:8,:]
  mini_3 = X_train.iloc[8:12,:]
  mini_4 = X_train.iloc[12:16,:]


In [11]:
#mini_batch()

In [12]:
# #제대로 되는지 테스트
# temp_w=mini_1 * weight #가중치 연산은 제대로 되는 것을 확인
# res = temp_w.apply(np.sum,axis = 1) + bias #연산도 제대로 확인
# res_sigmoid = res.apply(lambda x : 1 / (1 + np.exp(-x)) )
# res_sigmoid 

In [13]:
def forward_propa():
  #각 데이터를 가중치를 곱하고 bias를 더해서 리턴하기
  
  li = [mini_1,mini_2,mini_3,mini_4]
  tt_acc = []
  global tt_loss
  tt_loss = []

  for i in li:
    temp = i * weight 
    res = temp.apply(np.sum,axis = 1) + bias
    
    #가중치-편향 연산에 시그모이드 함수를 apply를 쓰기
    res_activated = res.apply(lambda x : 1 / (1 + np.exp(-x)) )
    tt_loss.append(res_activated.tolist())
  
    #조건문으로 0.5이상이면 1을 아니면 0을 리턴하기
    #ans = [1 if i >= 0.5 else 0 for i in res_activated]
    ans = []
    for i in res_activated:
      if i >= 0.5:
        ans.append(1)
      else:
        ans.append(0)
    
    
    tt_acc.append(ans)

  global predict_1,predict_2,predict_3,predict_4

  predict_1 = tt_acc[0]
  predict_2 = tt_acc[1]
  predict_3 = tt_acc[2]
  predict_4 = tt_acc[3]
  


In [14]:
#forward_propa()
#0인덱스 데이터 가중치-편향 연산 결과 


In [15]:
#정확도 산출하기

def acc():
  mean_1 = np.mean(y_train[0:4]   == predict_1)
  mean_2 = np.mean(y_train[4:8]   == predict_2)
  mean_3 = np.mean(y_train[8:12]  == predict_3)
  mean_4 = np.mean(y_train[12:16] == predict_4)

  global accuracy

  accuracy = np.mean((mean_1,mean_2,mean_3,mean_4))



In [16]:
#acc() #모델의 정확도 확인

L
=
−
1
N
∑
N
i
=
1
t
i
l
o
g
(
y
i
)
+
(
1
−
t
i
)
l
o
g
(
1
−
y
i
)
   ...(binary crossentropy)

yi는 예측값, ti는 실제값

In [17]:

def loss():
  global loss_cross1,loss_cross2,loss_cross3,loss_cross4
  loss_cross1,loss_cross2,loss_cross3,loss_cross4 = 0,0,0,0
  delta = 1e-7
  for i in range(4):
    temp1 = y_train[0:4].tolist()[i] * np.log(tt_loss[0][i]+delta)
    temp2 = y_train[4:8].tolist()[i] * np.log(tt_loss[1][i]+delta)
    temp3 = y_train[8:12].tolist()[i] * np.log(tt_loss[2][i]+delta)
    temp4 = y_train[12:16].tolist()[i] * np.log(tt_loss[3][i]+delta)

    loss_cross1 += temp1
    loss_cross2 += temp2
    loss_cross3 += temp3
    loss_cross4 += temp4


  global loss_cee
  loss_cee = -(loss_cross1 + loss_cross2 + loss_cross3 + loss_cross4) / 4
  #[Epoch 1] TrainData - Loss = 3.500, Accuracy = 0.800
  print(f'TrainData - Loss = {loss_cee}, Accuracy = {accuracy}') 


In [18]:
def main():
  load_data()
  para()
  data_shu()
  data_split()
  mini_batch()
  forward_propa()
  acc()
  loss()



In [19]:
main()

TrainData - Loss = 7.703449122378814, Accuracy = 0.8125
